# **Test communication with database** -In this notebook we going to examine the dataengine , ingest and dataset
**dataengine** - each dataengine object connect a data and metadata database to a holoclean project  
**ingest** - ingest file get a file and after finding the extension try to read it and put the data to the database and for now we just ingest a csv file to the database
**dataset** - This object keep the tables of one holoclean project and each dataengine create one of them for keep tracking the tables of that dataset when it goes through the algorithm 

Note that for step of this tutorial you need setup mysql server and have mysql-client application to access and create it . so first we need to create database and proper user.
in the database config file the data ordered as below 

1th line : db address (for now it is 'localhost') 

2th line : db name 

3th line : db user that have read and write privilages

4th line : user password (for test it is plain text but later we use RSA to hide it)

First we need to import modules

In [1]:
import sys
import os
sys.path.append('../')
from holoclean import dataengine , dataset 
from test import test_db_connection

## First step : Creating Dataset object
For first step we need a dataset that keep all tables information of the a holoclean project each holoclean project has a table T and information about cell which they are clean or not. At the first setting most of the this table are empty but eventually they will fill with data that comes from algorithm parts.

In [2]:
dataset_ins=dataset.Dataset()

For each dataset object we create a random id based on the time because in the database we might have many projects that running on the database the probability of creating same id for two project is $ 10^{-12} $ so we can trust on this randomness we can put data tables into the data and meta databases based on this id. 

In follow we print the id of the dataset_ins 

In [3]:
print(dataset_ins.dataset_id)

374388228355


We make unique table id for all tables in the dataset based on this dataset id which you can see below

In [4]:
print(dataset_ins.table_name)

['374388228355', '374388228355_T', '374388228355_C_clean', '374388228355_C_dk', '374388228355_X', '374388228355_D', '374388228355_Y', '374388228355_W', '374388228355_b', '374388228355_Y_prod', '374388228355_config']


After creating dataset now we can be sure all data from dataset can be retrieved from database.

## Second step : Creating Dataengine object 
Our next move is create connection between dataset to database , Dataengine is the connector of the dataset to the database , we have two different databases on for metadata database and other one is data database. the metadata database is just a database with one table that we called it 'metadata' and it stores the dataset table schema information. The other database stores the actual data , the intution behind this two database is if the process shutted down in the middle  of any step we can continue from the last check point which is the last data saving. 

In [5]:
dataengine_obj=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',dataset_ins)

By creating the this object all connection will create and pass it each db engine as members of dataengine object.after creating the connector we need to ingest the data to the database for this we should make ingest object.

## Third step : Ingest file to database

In this step we read data to our database for ingesting a data to a database we have method in dataengine which its name is ingest and it create reader object. Based on the file extension it calls proper class of reader to handel the reading. The ingest method need file address, for this version we just support csv files

We ingest that csv file with size of chunk( which it size by default is 20000), the intution behind chunk size is if we have very large data then we can ingest it and memory size wont be obstacle. 

In [6]:
dataengine_obj.ingest("10.csv")

For testing that data are in the data base we try to use dataengine_obj methods first we make a query and try to test the retrieve method , the query is just get all data from table 'T' of our dataset

In [7]:
sql_query="Select * from "+dataset_ins.table_name[1]
returned_dataframe=dataengine_obj.retrieve(sql_query)
print(returned_dataframe)

   level_0  index         city      date tempType  temp
0        0      1  ITE00100554  18000101     TMAX   -75
1        1      2  ITE00100554  18000101     TMIN  -148
2        2      3  GM000010962  18000101     PRCP     0
3        3      4  EZE00100082  18000101     TMAX   -86
4        4      5  EZE00100082  18000101     TMAX  -135
5        5      6  ITE00100554  18000102     TMAX   -60
6        6      7  ITE00100554  18000102     TMIN  -125
7        7      8  GM000010962  18000102     PRCP     0
8        8      9  EZE00100082  18000102     TMAX   -44
9        9     10  EZE00100082  18000102     TMIN  -135


We have another function which return schema of the dataset table from metadata database

In [8]:
dataengine_obj.get_schema("T")

'index,city,date,tempType,temp'

In the test directory we have two unittest that tests the correctness of adding data and retrive it from database got to the directory of the project and rune the following command 

python test_db_connection.py -v

In [9]:
x=test_db_connection.Dbtest
